# Cayley-Dickson Construction Applied to Qi Definition

*Version 1*

My original implementation of Gaussian integers included two classes, ``Zi`` and ``Qi``, where, for example, ``Zi(2, -7)`` represents a Gaussian integer, and ``Qi(-2/3, 4/5)`` represents a Gaussian rational.

I'd like to extend this code to include rational-valued quaternions and octonions using the Cayley-Dickson construction.

The [Cayley-Dickson construction](https://en.wikipedia.org/wiki/Cayley%E2%80%93Dickson_construction) is a process by which one can use a recursive definition of conjugation together with a recursive definition of multiplication to use...
* pairs of real numbers ($\mathbb{R}$) to create complex numbers,
* pairs of complex numbers ($\mathbb{C}$) to create quaternions,
* pairs of quaternions ($\mathbb{H}$) to create octonions,
* pairs of octonions ($\mathbb{O}$) to create sedenions ($\mathbb{S}$), and so on.

For more specifics, see my write-up about the Cayley-Dickson construction [at this link](https://abstract-algebra.readthedocs.io/en/latest/55_cayley_dickson.html).

In [1]:
# from cayley_dickson_alg import Zi, SetScalarMult
from random import randint

In [2]:
from random import seed

seed(42)  # Generate the same random sequence each time (for testing)

In [3]:
from fractions import Fraction
from functools import wraps

In [4]:
def gaussian_rational(fnc):
    """For use as a property that casts an argument into Gaussian rational."""
    @wraps(fnc)
    def gaussian_rational_wrapper(arg, num):
        qi = to_gaussian_rational(num)
        return fnc(arg, qi)
    return gaussian_rational_wrapper

In [5]:
class Qi:
    """Gaussian Rational Number Class"""

    __MAX_DENOMINATOR = 1_000_000

    def __init__(self, re=None, im=None):

        # --------------------------------------------------------
        # re is a str, float, int, or Fraction; and
        # im is a str, float, int, Fraction, or None

        if isinstance(re, (str, float, int, Fraction)):
            if isinstance(re, Fraction):
                self.__re = re
            else:
                self.__re = Fraction(re)
            if im is None:
                self.__im = 0
            elif isinstance(im, (str, float, int, Fraction)):
                if isinstance(im, Fraction):
                    self.__im = im
                else:
                    self.__im = Fraction(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --------------------------------------------------------
        # re is a complex; and
        # im is complex or None

        elif isinstance(re, complex):
            if im is None:
                self.__re = Fraction(re.real)
                self.__im = Fraction(re.imag)
            elif isinstance(im, complex):
                self.__re = Qi(re.real, re.imag)
                self.__im = Qi(im.real, im.imag)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --------------------------------------------------------
        # re is a Zi, and im is None, a complex, or a Zi

        elif isinstance(re, Zi):
            if im is None:
                self.__re = re.real
                self.__im = re.imag
            elif isinstance(im, (complex, Zi)):
                self.__re = Zi(re)
                self.__im = Zi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --------------------------------------------------------
        # re is a list or tuple of numbers with length equal to a
        # power of 2, and im is None, or it is a tuple or list
        # similar to the one input for re.
        elif isinstance(re, (tuple, list)):
            z = Zi.from_array(re)
            if im is None:
                self.__re = z.real
                self.__im = z.imag
            elif isinstance(im, (tuple, list)) and len(im) == len(re):
                w = Zi.from_array(im)
                self.__re = z
                self.__im = w
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --------------------------------------------------------
        # Both re and im are None

        elif re is None:
            self.__re = 0
            if im is None:
                self.__im = 0
            else:
                raise Exception(f"If re is None, then im must be None. But im = {im}")
        else:
            raise Exception(f"Unexpected combination of input types: {re} and {im}")

    @classmethod
    def max_denominator(cls, denom=None):
        if denom is None:
            return cls.__MAX_DENOMINATOR
        elif isinstance(denom, int) and denom > 1:
            cls.__MAX_DENOMINATOR = denom
            return cls.__MAX_DENOMINATOR
        else:
            raise ValueError(f"Maximum denominator, {denom}, must be an integer > 1")

    @property
    def real(self) -> Fraction:
        return self.__re

    @property
    def imag(self) -> Fraction:
        return self.__im

    def __repr__(self):
        return f"Qi({repr(str(self.__re))}, {repr(str(self.__im))})"

    def __str__(self):
        if self.__im < 0:
            return f"({self.__re}{self.__im}j)"
        else:
            return f"({self.__re}+{self.__im}j)"

    @gaussian_rational
    def __add__(self, other):
        return Qi(self.__re + other.real, self.__im + other.imag)

In [6]:
half = Fraction(1, 2)
two_thirds = Fraction(2, 3)
print(half, two_thirds)

1/2 2/3


In [7]:
print(f"{Qi(1) = }")
print(f"{Qi(1.5) = }")
print(f"{Qi(two_thirds) = }")
print(f"{Qi(1, 2) = }")
print(f"{Qi(1.5, 4.25) = }")
print(f"{Qi('1/2', '2/3') = }")
print(f"{Qi(half) = }")
print(f"{Qi('1/2', '2/3') = }")
print(f"{Qi(half, two_thirds) = }")

Qi(1) = Qi('1', '0')
Qi(1.5) = Qi('3/2', '0')
Qi(two_thirds) = Qi('2/3', '0')
Qi(1, 2) = Qi('1', '2')
Qi(1.5, 4.25) = Qi('3/2', '17/4')
Qi('1/2', '2/3') = Qi('1/2', '2/3')
Qi(half) = Qi('1/2', '0')
Qi('1/2', '2/3') = Qi('1/2', '2/3')
Qi((1-2j)) = Qi('1', '-2')
Qi(half, two_thirds) = Qi('1/2', '2/3')


In [12]:
print(f"{Qi((1-2j)) = }")

print(f"{Qi((1-2j), (-3+5j)) = }")

Qi((1-2j)) = Qi('1', '-2')
Qi((1-2j), (-3+5j)) = Qi('(1-2j)', '(-3+5j)')


In [9]:
Qi((1-2j), (-3+5j))

Qi('(1-2j)', '(-3+5j)')